In [15]:
import matplotlib.pyplot as plt
import numpy as np
import os
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from torch.utils.data import DataLoader

In [29]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.batchnorm1 = nn.BatchNorm2d(128)
        self.batchnorm2 = nn.BatchNorm2d(512)

        self.dropout = nn.Dropout2d(p=0.5)

        self.fc4 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.batchnorm1(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.batchnorm2(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        return x

In [30]:
data_directory = './data/Training'

transform = transforms.Compose([
    transforms.Resize((75, 50)),
    transforms.ToTensor(),
])

custom_dataset = ImageFolder(root=data_directory, transform=transform)

In [31]:
torch.manual_seed(42)

In [32]:
device = "cpu"

In [33]:
batch_size = 32
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [34]:
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [35]:
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Print epoch and loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

# Save your trained model if needed
torch.save(model.state_dict(), 'your_model.pth')

RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1

In [36]:
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(75, 50, 3)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))  # binary classification

In [37]:
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [38]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 73, 48, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 36, 24, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 34, 22, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 17, 11, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 15, 9, 128)        73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 7, 4, 128)         0

In [41]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    './data/Training',
    target_size=(75, 50),  # Adjust based on your model's input shape
    batch_size=32,
    class_mode='binary'  # Use 'categorical' for multi-class classification
)

validation_generator = test_datagen.flow_from_directory(
    './data/Testing',
    target_size=(75, 50),
    batch_size=32,
    class_mode='binary'
)

Found 144 images belonging to 2 classes.
Found 96 images belonging to 2 classes.


In [43]:
model2.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,  # Adjust the number of epochs based on your needs
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

C:\Users\user\AppData\Local\Temp\ipykernel_1320\92103231.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2.fit_generator(


Epoch 1/10
5/5 [==============================] - 3s 347ms/step - loss: 0.6986 - accuracy: 0.5694 - val_loss: 0.6789 - val_accuracy: 0.5833
Epoch 2/10
5/5 [==============================] - 1s 181ms/step - loss: 0.6782 - accuracy: 0.5833 - val_loss: 0.6798 - val_accuracy: 0.5833
Epoch 3/10
5/5 [==============================] - 1s 173ms/step - loss: 0.6806 - accuracy: 0.5833 - val_loss: 0.6802 - val_accuracy: 0.5833
Epoch 4/10
5/5 [==============================] - 1s 191ms/step - loss: 0.6864 - accuracy: 0.5833 - val_loss: 0.6812 - val_accuracy: 0.5833
Epoch 5/10
5/5 [==============================] - 1s 173ms/step - loss: 0.6809 - accuracy: 0.5833 - val_loss: 0.6807 - val_accuracy: 0.5833
Epoch 6/10
5/5 [==============================] - 1s 173ms/step - loss: 0.6726 - accuracy: 0.5833 - val_loss: 0.6804 - val_accuracy: 0.5833
Epoch 7/10
5/5 [==============================] - 1s 173ms/step - loss: 0.6874 - accuracy: 0.5833 - val_loss: 0.6882 - val_accuracy: 0.5833
Epoch 8/10
5/5 [====